In [1]:
## some experiments gave rectifiedAdam not found error: Therefore run again if required
PARAMS_COMMON={
    'project_from':'HYBRID', #balanced
    'project_to':'RefinedTreshold-Hybrid-Test',
    'tags':['samplewise', 'balanced']
}

In [2]:
import sys
sys.path.append('/home/ubuntu/intelliscope/modules')
sys.path.append('../..')


#!pip install neptune-client
import os
import neptune

import tensorflow as tf
from tensorflow.keras.models import load_model

import matplotlib.pyplot as plt
import cv2
import numpy as np
from tensorflow import keras
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, Lambda,BatchNormalization
from tensorflow.keras import backend as K
from tensorflow_addons.layers import InstanceNormalization
import tensorflow_addons as tfa
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, Lambda
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from tensorflow_addons.optimizers import RectifiedAdam


from intelliscope import instead_data_loaders
from sklearn.metrics import classification_report
import numpy as np
from neptunecontrib.monitoring.metrics import *
from neptune.sessions import Session
import custom_metrics


NEPTUNE_API_TOKEN='eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vdWkubmVwdHVuZS5haSIsImFwaV91cmwiOiJodHRwczovL3VpLm5lcHR1bmUuYWkiLCJhcGlfa2V5IjoiMzc1YTM5OGMtYTY3Ny00ZmM4LTg5ZGQtOGI2YTQ1YmZiMDkzIn0='
session = Session(api_token=NEPTUNE_API_TOKEN)

project = session.get_projects('intelliscope')['intelliscope/'+PARAMS_COMMON['project_from']]
projects=project.get_experiments()



def get_results(model=None, X_pcg=None, X_ecg=None, Y=None):    
    y_pred = model.predict([X_pcg,X_ecg], batch_size=32, verbose=1)
    y = np.argmax(Y,axis=1)
    
    return y, y_pred
    


def neptune_log_metrics(y_test=None, y_pred=None, threshold = 0.5):
    log_confusion_matrix(y_test, y_pred[:, 1] > threshold)
    log_classification_report(y_test, y_pred[:, 1] > threshold)
    log_class_metrics(y_test, y_pred[:, 1] > threshold)
    log_class_metrics_by_threshold(y_test, y_pred[:, 1])
    log_brier_loss(y_test, y_pred[:, 1])
    log_prediction_distribution(y_test, y_pred[:, 1])

    log_log_loss(y_test, y_pred)
    log_roc_auc(y_test, y_pred)
    log_precision_recall_auc(y_test, y_pred)
    log_ks_statistic(y_test, y_pred)
    log_cumulative_gain(y_test, y_pred)
    log_lift_curve(y_test, y_pred)
    

    

In [3]:
project_name=PARAMS_COMMON['project_to']
X_val,Y_val = instead_data_loaders(projects[0].get_parameters(),'val')
X_test,Y_test = instead_data_loaders(projects[0].get_parameters(),'test')

Total  194
101 Images loaded across 2 Categories. Ignored blanks  0
93 Images loaded across 2 Categories. Ignored blanks  0
194 Total After removing blanks
Total  388
197 Images loaded across 2 Categories. Ignored blanks  0
191 Images loaded across 2 Categories. Ignored blanks  16
372 Total After removing blanks


In [4]:
from sklearn.metrics import roc_curve
from numpy import sqrt
from numpy import argmax



def get_optimal_thres(model, y_val, y_val_pred):
    
    y_val_pred_1=y_val_pred[:,1] 
    fpr, tpr, thresholds = roc_curve(y_val, y_val_pred_1)
    gmeans = sqrt(tpr * (1-fpr))
    ix = argmax(gmeans)
    
    print(f'\nfpr : {fpr[ix]} , tpr : {tpr[ix]} , gmeans : {gmeans[ix]}')
    return thresholds[ix]
    

In [5]:
X_test_pcg = X_test[:,:,:,0:3]/255.0
X_test_ecg = X_test[:,:,:,3:6]/255.0

X_val_pcg = X_val[:,:,:,0:3]/255.0
X_val_ecg = X_val[:,:,:,3:6]/255.0

for i in range(len(projects)):
        
    project    = projects[i]
    print('\n\n',str(project))
    
    model_name = str(project)+'.h5'
    PARAMS     = project.get_parameters()
    PARAMS['tags']=PARAMS['tags'][2:-2].strip().split("', '")+PARAMS_COMMON['tags']
    
    model = load_model(PARAMS['modelsave_dir']+'/'+model_name)
    
    y_val, y_val_pred=get_results(model, X_val_pcg, X_val_ecg, Y_val)
    
    optimal_thres=get_optimal_thres(model, y_val, y_val_pred)
    
    print('optimal_threshold : ',  optimal_thres, '\n')
    y_test, y_test_pred=get_results(model, X_test_pcg, X_test_ecg, Y_test)
    
    
    sk_metrics=custom_metrics._class_metrics(y_test, y_test_pred[:, 1] > optimal_thres)
    print(sk_metrics)
    gmean=sqrt(sk_metrics['true_negative_rate']*sk_metrics['true_positive_rate'])
    
    PARAMS['G-Mean']=gmean 
    PARAMS['optimal_threshold']=optimal_thres
    
    neptune.init('intelliscope/'+PARAMS_COMMON['project_to'],NEPTUNE_API_TOKEN)
    exp = neptune.create_experiment(name=str(project),description=PARAMS['name'],params=PARAMS,tags=PARAMS['tags'],upload_stdout=True)
    neptune_log_metrics(y_test, y_test_pred, optimal_thres)
    neptune.stop()



 Experiment(HYBRID-1)
194/194 [==============================] - 1s 4ms/sample

fpr : 0.45544554455445546 , tpr : 0.3978494623655914 , gmeans : 0.46545751398790125
optimal_threshold :  0.50952107 

372/372 [==============================] - 1s 4ms/sample
{'accuracy': 0.5698924731182796, 'precision': 0.5358851674641149, 'recall': 0.64, 'f1_score': 0.5833333333333334, 'f2_score': 0.6160616061606161, 'matthews_corrcoef': 0.1484955257047406, 'cohen_kappa': 0.14603001520847092, 'true_positive_rate': 0.64, 'true_negative_rate': 0.5076142131979695, 'positive_predictive_value': 0.5358851674641149, 'negative_predictive_value': 0.6134969325153374, 'false_positive_rate': 0.49238578680203043, 'false_negative_rate': 0.36, 'false_discovery_rate': 0.46411483253588515}


NVMLError: NVML Shared Library Not Found - GPU usage metrics may not be reported.


https://ui.neptune.ai/intelliscope/RefinedTreshold-Hybrid-Test/e/REF1-34


 Experiment(HYBRID-11)
194/194 [==============================] - 1s 4ms/sample

fpr : 0.25742574257425743 , tpr : 0.7526881720430108 , gmeans : 0.7476141119775483
optimal_threshold :  0.5987696 

372/372 [==============================] - 1s 4ms/sample
{'accuracy': 0.75, 'precision': 0.75, 'recall': 0.7028571428571428, 'f1_score': 0.7256637168141593, 'f2_score': 0.7118055555555556, 'matthews_corrcoef': 0.4973606917750815, 'cohen_kappa': 0.496477848285498, 'true_positive_rate': 0.7028571428571428, 'true_negative_rate': 0.7918781725888325, 'positive_predictive_value': 0.75, 'negative_predictive_value': 0.75, 'false_positive_rate': 0.20812182741116753, 'false_negative_rate': 0.29714285714285715, 'false_discovery_rate': 0.25}


https://ui.neptune.ai/intelliscope/RefinedTreshold-Hybrid-Test/e/REF1-35


 Experiment(HYBRID-12)
194/194 [==============================] - 1s 4ms/sample

fpr : 0.12871287128712872 , tpr : 0.7634408602150538 , gmeans : 0.8155833464697883
optimal_threshold :  0.12357287 

372/372 [==============================] - 1s 4ms/sample
{'accuracy': 0.782258064516129, 'precision': 0.7582417582417582, 'recall': 0.7885714285714286, 'f1_score': 0.7731092436974789, 'f2_score': 0.782312925170068, 'matthews_corrcoef': 0.564362393045586, 'cohen_kappa': 0.5639615651771244, 'true_positive_rate': 0.7885714285714286, 'true_negative_rate': 0.7766497461928934, 'positive_predictive_value': 0.7582417582417582, 'negative_predictive_value': 0.8052631578947368, 'false_positive_rate': 0.2233502538071066, 'false_negative_rate': 0.21142857142857144, 'false_discovery_rate': 0.24175824175824176}


https://ui.neptune.ai/intelliscope/RefinedTreshold-Hybrid-Test/e/REF1-36


 Experiment(HYBRID-13)
194/194 [==============================] - 1s 4ms/sample

fpr : 0.22772277227722773 , tpr : 0.8064516129032258 , gmeans : 0.7891794573514072
optimal_threshold :  0.4914377 

372/372 [==============================] - 1s 4ms/sample
{'accuracy': 0.7365591397849462, 'precision': 0.6824644549763034, 'recall': 0.8228571428571428, 'f1_score': 0.7461139896373058, 'f2_score': 0.7903402854006586, 'matthews_corrcoef': 0.4863235606376831, 'cohen_kappa': 0.47727337902555134, 'true_positive_rate': 0.8228571428571428, 'true_negative_rate': 0.6598984771573604, 'positive_predictive_value': 0.6824644549763034, 'negative_predictive_value': 0.8074534161490683, 'false_positive_rate': 0.3401015228426396, 'false_negative_rate': 0.17714285714285713, 'false_discovery_rate': 0.3175355450236967}


...

https://ui.neptune.ai/intelliscope/RefinedTreshold-Hybrid-Test/e/REF1-52


 Experiment(HYBRID-29)
194/194 [==============================] - 1s 4ms/sample

fpr : 0.19801980198019803 , tpr : 0.7741935483870968 , gmeans : 0.7879643997295418
optimal_threshold :  0.5357406 

372/372 [==============================] - 1s 4ms/sample
{'accuracy': 0.7204301075268817, 'precision': 0.7100591715976331, 'recall': 0.6857142857142857, 'f1_score': 0.6976744186046512, 'f2_score': 0.6904487917146146, 'matthews_corrcoef': 0.43805195945502956, 'cohen_kappa': 0.4378215001889041, 'true_positive_rate': 0.6857142857142857, 'true_negative_rate': 0.751269035532995, 'positive_predictive_value': 0.7100591715976331, 'negative_predictive_value': 0.729064039408867, 'false_positive_rate': 0.24873096446700507, 'false_negative_rate': 0.3142857142857143, 'false_discovery_rate': 0.28994082840236685}


https://ui.neptune.ai/intelliscope/RefinedTreshold-Hybrid-Test/e/REF1-53


 Experiment(HYBRID-30)
194/194 [==============================] - 1s 4ms/sample

fpr : 0.3069306930693069 , tpr : 0.7526881720430108 , gmeans : 0.7222638504956341
optimal_threshold :  0.99999666 

372/372 [==============================] - 1s 4ms/sample
{'accuracy': 0.7150537634408602, 'precision': 0.7065868263473054, 'recall': 0.6742857142857143, 'f1_score': 0.6900584795321637, 'f2_score': 0.6805074971164936, 'matthews_corrcoef': 0.42704379271163545, 'cohen_kappa': 0.4266437898042865, 'true_positive_rate': 0.6742857142857143, 'true_negative_rate': 0.751269035532995, 'positive_predictive_value': 0.7065868263473054, 'negative_predictive_value': 0.7219512195121951, 'false_positive_rate': 0.24873096446700507, 'false_negative_rate': 0.32571428571428573, 'false_discovery_rate': 0.2934131736526946}


NVMLError: NVML Shared Library Not Found - GPU usage metrics may not be reported.


https://ui.neptune.ai/intelliscope/RefinedTreshold-Hybrid-Test/e/REF1-54


 Experiment(HYBRID-31)


ValueError: Unknown optimizer: RectifiedAdam